**Note: This is an example for how to train the model. Model performance and some setting are not same as we train our model. We recommend you to read our paper  and change parameter setting to be the same.**

In [1]:
from model import create_model, tilted_loss
from preprocess_input import smooth, buildData
import pandas as pd
from keras import optimizers, callbacks

Using TensorFlow backend.


## Data preprocess

In [0]:
data_2018 = pd.read_csv('./example/example_2018.csv',header=None).values
data_2019 = pd.read_csv('./example/example_2019.csv',header=None).values

In [0]:
data_2018 = data_2018.reshape(-1,85,1)
data_2019 = data_2019.reshape(-1,85,1)

In [0]:
X, X2, y = buildData(data_2019,data_2018)

In [0]:
X = smooth(X)
X2 = smooth(X2)

In [6]:
X.shape, X2.shape, y.shape

((76, 1190, 3), (76, 85, 3), (76, 85, 1))

In [0]:
split_ratio = 0.7
length = X.shape[0]
split = int(split_ratio*length)

X_train = X[:split]
X_train2 = X2[:split]
y_train = y[:split]

X_eval = X[split:]
X_eval2 = X2[split:]
y_eval = y[split:]

## Create model

In [0]:
encoder_input_length =  14*85 #day*timestep
decoder_input_length = 85 #timestep, output length will be the same as decoder input length
quantile = 0.33

model = create_model(encoder_input_length,decoder_input_length)
model.compile(optimizer=optimizers.adam(lr=0.00005, decay=0.000001), loss=lambda y,y_pred: tilted_loss(quantile,y,y_pred))

In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1190, 3)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1190, 512)    532480      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 85, 3)        0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 787456      bidirectional_1[0][0]            
____________________________________________________________________________________________

In [0]:
file_path = './weights/seq2seq_14.h5'

sb = callbacks.ModelCheckpoint(file_path, save_best_only=True, verbose=1, monitor="val_loss", mode="min")

In [12]:
history = model.fit([X_train,X_train2], y_train, epochs=20, batch_size=128, verbose=1, 
          validation_data=([X_eval,X_eval2],y_eval),callbacks=[sb])

Train on 53 samples, validate on 23 samples
Epoch 1/20
53/53 [==============================] - 24s 459ms/step - loss: 0.0968 - val_loss: 0.0922

Epoch 00001: val_loss improved from inf to 0.09216, saving model to ./weights/seq2seq_14.h5
Epoch 2/20
53/53 [==============================] - 27s 506ms/step - loss: 0.0965 - val_loss: 0.0916

Epoch 00002: val_loss improved from 0.09216 to 0.09155, saving model to ./weights/seq2seq_14.h5
Epoch 3/20
53/53 [==============================] - 22s 411ms/step - loss: 0.0960 - val_loss: 0.0909

Epoch 00003: val_loss improved from 0.09155 to 0.09091, saving model to ./weights/seq2seq_14.h5
Epoch 4/20
53/53 [==============================] - 22s 410ms/step - loss: 0.0955 - val_loss: 0.0903

Epoch 00004: val_loss improved from 0.09091 to 0.09025, saving model to ./weights/seq2seq_14.h5
Epoch 5/20
53/53 [==============================] - 22s 412ms/step - loss: 0.0936 - val_loss: 0.0896

Epoch 00005: val_loss improved from 0.09025 to 0.08958, saving mod